In [1]:
import pandas as pd
import numpy as np

#File to load
file = "Seasons_stats.csv"

#Reading file and storing to dataframe
nba_data = pd.read_csv(file)

nba_data.head()

#Used to display all columns
#for col in nba_data.columns: 
    #print(col)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,6,7,8,9,10,13,14,15,16,17,18,19,20,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Key,NaN,NaN,NaN,NaN,NaN,Games,Games Started,Minutes Played,Player Efficency Rating,...,Free Throw Percentage,offensive rebounds,defensive rebounds,total rebounds,assists,steals,Blocks,Turnovers,Personal Fouls,Points
1,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176,NaN,NaN,NaN,217,458
2,1,1950.0,Cliff Barker,SG,29.0,INO,49,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109,NaN,NaN,NaN,99,279
3,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140,NaN,NaN,NaN,192,438
4,3,1950.0,Ed Bartels,F,24.0,TOT,15,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20,NaN,NaN,NaN,29,63


**Adding Minutes/Game Calculation to the CSV**

In [70]:
#Filtering data to specific years and grouping by player
player_data = nba_data.loc[nba_data['Year'] >= 1980]

#Dropping NA/NaN values throughout the CSV
player_data.dropna(how='any')

#Setting the G, MP, and GS columns equal to non-NA/NaN values only
player_data = player_data[player_data['G'].notna()]
player_data = player_data[player_data['MP'].notna()]
player_data = player_data[player_data['GS'].notna()]

#Converting the G, MP columns to numberic dtype
player_data['G'] = pd.to_numeric(player_data['G'])
player_data['MP'] = pd.to_numeric(player_data['MP'])
player_data['FGA'] = pd.to_numeric(player_data['FGA'])

#Making new variables for G, MP
games = player_data['G']
minutes = player_data['MP']
FGA = player_data['FGA']

#Calculation for new column called Minutes/Game
minutes_per_game = minutes/games
FGA_per_game = FGA/games

#Adding Minutes/Game column to the CSV, mapping the column, converting to numeric dtype, applying comparison statement
player_data['Minutes/Game'] = minutes_per_game
player_data['Minutes/Game'] = player_data['Minutes/Game'].map('{:.1f}'.format)
player_data['Minutes/Game'] = pd.to_numeric(player_data['Minutes/Game'])
player_data = player_data.loc[(player_data['Minutes/Game'] >= 20) & (player_data['G'] >= 30)]
player_data['AST'] = pd.to_numeric(player_data['AST'])
player_data['APG'] = player_data['AST'] / player_data['G']
player_data['ORB'] = pd.to_numeric(player_data['ORB'])
player_data['ORBG'] = player_data['ORB'] / player_data['G']
player_data['3P'] = pd.to_numeric(player_data['3P'])
player_data['3PG'] = player_data['3P'] / player_data['G']

#Adding FGA/Game column to the dataframe
player_data['FGA/Game'] = FGA_per_game
player_data['FGA/Game'] = player_data['FGA/Game'].map('{:.1f}'.format)
player_data['FGA/Game'] = pd.to_numeric(player_data['FGA/Game'])

player_data.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,STL,BLK,TOV,PF,PTS,Minutes/Game,APG,ORBG,3PG,FGA/Game
5731,5730,1980.0,Tiny Archibald*,PG,31.0,BOS,80.0,80,2864.0,15.3,...,106,10,242,218,1131,35.8,8.387500,0.737500,0.050000,9.9
5749,5748,1980.0,Larry Bird*,PF,23.0,BOS,82.0,82,2955.0,20.5,...,143,53,263,279,1745,36.0,4.512195,2.634146,0.707317,17.8
5782,5781,1980.0,M.L. Carr,SF,29.0,BOS,82.0,7,1994.0,14.6,...,120,36,143,214,914,24.3,1.902439,1.292683,0.146341,9.3
5803,5802,1980.0,Dave Cowens*,C,31.0,BOS,66.0,55,2159.0,14.8,...,69,61,108,216,940,32.7,3.121212,1.909091,0.015152,14.1
5836,5835,1980.0,Chris Ford,SG,31.0,BOS,73.0,73,2115.0,13,...,111,27,105,178,816,29.0,2.945205,1.054795,0.958904,9.7


**Creating dataframe to determine top ten points in a season and points per minute played**

In [62]:
#Creating a series from the CSV with Year, Player, PTS, and our new column Minutes/Game
player_points = player_data[['Year','Player', 'PTS', 'Minutes/Game']]

#Converting the PTS column to a numberica dtype
player_points['PTS'] = pd.to_numeric(player_points['PTS'])

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_points = player_points.sort_values(by='PTS', ascending=False)
top_points.head(10)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Year,Player,PTS,Minutes/Game
8470,1987.0,Michael Jordan*,3041.0,40.0
8870,1988.0,Michael Jordan*,2868.0,40.4
17743,2006.0,Kobe Bryant,2832.0,41.0
9748,1990.0,Michael Jordan*,2753.0,39.0
9298,1989.0,Michael Jordan*,2633.0,40.2
22439,2014.0,Kevin Durant,2593.0,38.5
10213,1991.0,Michael Jordan*,2580.0,37.0
24655,2017.0,Russell Westbrook,2558.0,34.6
6558,1982.0,George Gervin*,2551.0,35.7
11107,1993.0,Michael Jordan*,2541.0,39.3


# Offensive Stat 1 - Points/Minutes Played (sorted by PTS)

In [63]:
#Calculation, creation, mapping of new column called Points/Min
points_per_game = player_points['PTS'] / player_data['G']
player_points['PTS/Game'] = points_per_game
player_points['PTS/Game'] = player_points['PTS/Game'].map('{:.1f}'.format)

#Displaying the final calculation
top_points_min = player_points.sort_values(by='PTS', ascending=False)
top_points_min.head(10)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Year,Player,PTS,Minutes/Game,PTS/Game
8470,1987.0,Michael Jordan*,3041.0,40.0,37.1
8870,1988.0,Michael Jordan*,2868.0,40.4,35.0
17743,2006.0,Kobe Bryant,2832.0,41.0,35.4
9748,1990.0,Michael Jordan*,2753.0,39.0,33.6
9298,1989.0,Michael Jordan*,2633.0,40.2,32.5
22439,2014.0,Kevin Durant,2593.0,38.5,32.0
10213,1991.0,Michael Jordan*,2580.0,37.0,31.5
24655,2017.0,Russell Westbrook,2558.0,34.6,31.6
6558,1982.0,George Gervin*,2551.0,35.7,32.3
11107,1993.0,Michael Jordan*,2541.0,39.3,32.6


In [64]:
#Creating a series from the CSV with new shooting percentages
player_initial_TS = player_data[['Year','Player','TS%', 'FGA/Game']]

#Converting the PTS column to a numberica dtype
player_initial_TS['TS%'] = pd.to_numeric(player_initial_TS['TS%'])

# #Calculation, creation, mapping of new column called Points/Min
# FGA_per_game = player_data['FGA'] / player_data['G']
# player_TS['FGA/Game'] = FGA_per_game
player_initial_TS['FGA/Game'] = player_initial_TS['FGA/Game'].map('{:.1f}'.format)

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_TS = player_initial_TS.sort_values(by='TS%', ascending=False)
top_TS.head(10)




/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Year,Player,TS%,FGA/Game
21219,2012.0,Tyson Chandler,0.708,5.7
24192,2017.0,Tyson Chandler,0.703,4.9
6559,1982.0,Artis Gilmore*,0.702,10.2
23197,2015.0,Kyle Korver,0.699,8.0
20628,2011.0,Tyson Chandler,0.697,5.5
22972,2015.0,Tyson Chandler,0.697,5.9
12553,1996.0,Tim Legler,0.688,6.0
24286,2017.0,Rudy Gobert,0.682,7.7
8003,1986.0,Darryl Dawkins,0.680,8.6
7661,1985.0,Artis Gilmore*,0.680,10.5


# Offensive Stat 2 - True Shooting Percentage - To be deleted in favor of line 32

In [27]:
'''
#Creating a series from the CSV with new shooting percentages
player_TS = player_data.loc[player_data['FGA/Game'] >= 10] #& (player_data['G'] >= 30)]
player_TS = player_TS[['Year','Player','TS%', 'FGA/Game', 'G']]

#Converting the PTS column to a numberica dtype
player_TS['TS%'] = pd.to_numeric(player_TS['TS%'])

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_TS = player_TS.sort_values(by='TS%', ascending=False)
top_TS.head(10)
'''

,Year,Player,TS%,FGA/Game,G
6559,1982.0,Artis Gilmore*,0.702,10.2,82.0
10102,1991.0,Johnny Dawkins,0.687,10.2,4.0
7661,1985.0,Artis Gilmore*,0.680,10.5,81.0
23634,2016.0,Stephen Curry,0.669,20.2,79.0
6939,1983.0,Artis Gilmore*,0.668,10.8,82.0
8700,1988.0,Charles Barkley*,0.665,16.0,80.0
6896,1983.0,Adrian Dantley*,0.661,18.3,22.0
9565,1990.0,Charles Barkley*,0.661,14.9,79.0
21339,2012.0,James Harden,0.660,10.1,62.0
8317,1987.0,Charles Barkley*,0.660,13.8,68.0


In [65]:
#Creating a series from the CSV with new shooting percentages
player_TS = player_data.loc[player_data['FGA/Game'] >= 10]
player_TS = player_TS[['Year','Player','TS%', 'FGA/Game', 'G']]

#Converting the PTS column to a numberica dtype
player_TS['TS%'] = pd.to_numeric(player_TS['TS%'])

#Sorting and displaying to check if the Minutes/Game column is calculating correctly
top_TS = player_TS.sort_values(by='TS%', ascending=False)
top_TS.head(10)

,Year,Player,TS%,FGA/Game,G
6559,1982.0,Artis Gilmore*,0.702,10.2,82.0
7661,1985.0,Artis Gilmore*,0.680,10.5,81.0
23634,2016.0,Stephen Curry,0.669,20.2,79.0
6939,1983.0,Artis Gilmore*,0.668,10.8,82.0
8700,1988.0,Charles Barkley*,0.665,16.0,80.0
9565,1990.0,Charles Barkley*,0.661,14.9,79.0
8317,1987.0,Charles Barkley*,0.660,13.8,68.0
21339,2012.0,James Harden,0.660,10.1,62.0
8911,1988.0,Kevin McHale*,0.656,14.2,64.0
19242,2008.0,Amar'e Stoudemire,0.656,15.3,79.0


# Offensive Stat 3 - Effective Field Goal Percentage (excludes free throws)

In [66]:
#Creating a dataframe from the CSV with new effective field goal percentage with more than attempts per game
player_efg = player_data.loc[player_data['FGA/Game'] >= 10]
player_efg['eFG%'] = pd.to_numeric(player_efg['eFG%'])
player_efg = player_efg.sort_values(by='eFG%', ascending=False)
player_efg = player_efg[['Year','Player', 'eFG%', 'G', 'FGA', 'FGA/Game']]
player_efg.head(10)

/Users/scotttheener/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Player,eFG%,G,FGA,FGA/Game
6559,1982.0,Artis Gilmore*,0.653,82.0,837.0,10.2
23634,2016.0,Stephen Curry,0.630,79.0,1598.0,20.2
6939,1983.0,Artis Gilmore*,0.626,82.0,888.0,10.8
7661,1985.0,Artis Gilmore*,0.623,81.0,854.0,10.5
18563,2007.0,Steve Nash,0.613,76.0,971.0,12.8
20168,2010.0,Dwight Howard,0.612,82.0,834.0,10.2
22549,2014.0,LeBron James,0.610,77.0,1353.0,17.6
19742,2009.0,Shaquille O'Neal*,0.609,75.0,841.0,11.2
24539,2017.0,Otto Porter,0.608,80.0,803.0,10.0
9565,1990.0,Charles Barkley*,0.608,79.0,1177.0,14.9


# Offensive Stat 4 - Assits Per Game Played

In [67]:
player_apg = player_data[['Year','Player', 'APG', 'AST', 'G']]
player_apg = player_apg.sort_values(by='APG', ascending=False)
player_apg['APG'] = player_apg['APG'].map('{:.1f}'.format)
player_apg.head(10)


,Year,Player,APG,AST,G
9924,1990.0,John Stockton*,14.5,1134.0,78.0
10378,1991.0,John Stockton*,14.2,1164.0,82.0
7858,1985.0,Isiah Thomas*,13.9,1123.0,81.0
9019,1988.0,John Stockton*,13.8,1128.0,82.0
10845,1992.0,John Stockton*,13.7,1126.0,82.0
9462,1989.0,John Stockton*,13.6,1118.0,82.0
7349,1984.0,Magic Johnson*,13.1,875.0,67.0
9284,1989.0,Magic Johnson*,12.8,988.0,77.0
8083,1986.0,Magic Johnson*,12.6,907.0,72.0
11778,1994.0,John Stockton*,12.6,1031.0,82.0


# Offensive Stat 5 - Offensive Rebound Per Game Played

In [69]:
player_orbg = player_data[['Year','Player', 'ORBG', 'ORB', 'G']]
player_orbg = player_orbg.sort_values(by='ORBG', ascending=False)
player_orbg.head(10)

,Year,Player,ORBG,ORB,G
6673,1982.0,Moses Malone*,6.888889,558.0,81.0
13942,1998.0,Jayson Williams,6.815385,443.0,65.0
10802,1992.0,Dennis Rodman*,6.378049,523.0,82.0
11249,1993.0,Dennis Rodman*,5.919355,367.0,62.0
13393,1997.0,Jayson Williams,5.902439,242.0,41.0
13279,1997.0,Dennis Rodman*,5.818182,320.0,55.0
8317,1987.0,Charles Barkley*,5.735294,390.0,68.0
11738,1994.0,Dennis Rodman*,5.734177,453.0,79.0
7046,1983.0,Moses Malone*,5.705128,445.0,78.0
12186,1995.0,Dennis Rodman*,5.591837,274.0,49.0


# Offensive Stat 6 - Three Pointers Made Per Game Played

In [75]:
player_3pg = player_data[['Year','Player', '3PG', '3P', '3PA', '3P%', 'G']]
player_3pg = player_3pg.sort_values(by='3PG', ascending=False)
player_3pg.head(10)

,Year,Player,3PG,3P,3PA,3P%,G
23634,2016.0,Stephen Curry,5.088608,402.0,886,0.454,79.0
24217,2017.0,Stephen Curry,4.101266,324.0,789,0.411,79.0
23001,2015.0,Stephen Curry,3.575000,286.0,646,0.443,80.0
21799,2013.0,Stephen Curry,3.487179,272.0,600,0.453,78.0
24034,2016.0,Klay Thompson,3.450000,276.0,650,0.425,80.0
17666,2006.0,Ray Allen,3.448718,269.0,653,0.412,78.0
24618,2017.0,Klay Thompson,3.435897,268.0,647,0.414,78.0
22403,2014.0,Stephen Curry,3.346154,261.0,615,0.424,78.0
15513,2002.0,Ray Allen,3.318841,229.0,528,0.434,69.0
24291,2017.0,Eric Gordon,3.280000,246.0,661,0.372,75.0
